<a href="https://colab.research.google.com/github/mille055/AIPI540_individual_project/blob/main/notebooks/AIPI540_IP_fusion_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!git clone 'https://github.com/mille055/AIPI540_individual_project.git'

Cloning into 'AIPI540_individual_project'...
remote: Enumerating objects: 316, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 316 (delta 10), reused 15 (delta 7), pack-reused 291
Receiving objects: 100% (316/316), 91.88 MiB | 18.16 MiB/s, done.
Resolving deltas: 100% (146/146), done.
Updating files: 100% (35/35), done.


In [11]:
!pip install -r https://raw.githubusercontent.com/mille055/AIPI540_individual_project/main/requirements.txt

ERROR: git+git@gitlab.oit.duke.edu:mille055/dicomtools.git@9339eaf549808a7935a133b849ee5a37ccf8086a#egg=dicomtools is not a valid editable requirement. It should either be a path to a local project or a VCS URL (beginning with bzr+http, bzr+https, bzr+ssh, bzr+sftp, bzr+ftp, bzr+lp, bzr+file, git+http, git+https, git+ssh, git+git, git+file, hg+file, hg+http, hg+https, hg+ssh, hg+static-http, svn+ssh, svn+http, svn+https, svn+svn, svn+file).


In [15]:
!pip install pydicom==2.1.2
!pip install monai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.4 MB/s eta 0:00:00


In [13]:
!pip show pydicom

Name: pydicom
Version: 2.1.2
Summary: Pure python package for DICOM medical file reading and writing
Home-page: https://github.com/pydicom/pydicom
Author: Darcy Mason and contributors
Author-email: darcymason@gmail.com
License: MIT
Location: /usr/local/lib/python3.9/dist-packages
Requires: 
Required-by: 


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import torchvision
import pydicom
import monai
import pickle

from torchvision import datasets, models, transforms


In [17]:
#local imports
from AIPI540_individual_project.scripts.config import *
# from AIPI540_individual_project.scripts.train_pixel_model import train_model
# from AIPI540_individual_project.scripts.train_text_model import load_text_data, train_text_model, list_incorrect_text_predictions
# from AIPI540_individual_project.scripts.utils import *

# Load Datasets


In [18]:
train_datafile = '/content/AIPI540_individual_project/data/X_train02282023.pkl'
test_datafile = '/content/AIPI540_individual_project/data/X_test02282023.pkl'

In [19]:
def load_dataset(train_file, test_file):
  with open(train_file, 'rb') as f:
    train_df = pickle.load(f)
  with open(test_file, 'rb') as g:
    test_df = pickle.load(g)

  return train_df, test_df

In [20]:
train, test = load_dataset(train_datafile, test_datafile)

In [21]:
train.columns.tolist()

['fname_x',
 'StudyInstanceUID',
 'SeriesInstanceUID',
 'SeriesNumber',
 'SeriesDescription_x',
 'AcquisitionNumber',
 'InstanceNumber',
 'ImageOrientationPatient',
 'MRAcquisitionType',
 'AngioFlag',
 'SliceThickness',
 'RepetitionTime',
 'EchoTime',
 'EchoTrainLength',
 'PixelSpacing',
 'ContrastBolusAgent',
 'InversionTime',
 'seq_E',
 'seq_EP',
 'seq_G',
 'seq_IR',
 'seq_P',
 'seq_R',
 'seq_S',
 'seq_SE',
 'var_E',
 'var_K',
 'var_N',
 'var_O',
 'var_OSP',
 'var_P',
 'var_S',
 'var_SK',
 'var_SP',
 'var_SS',
 'opt_2',
 'opt_A',
 'opt_ACC_GEMS',
 'opt_D',
 'opt_EDR_GEMS',
 'opt_EPI_GEMS',
 'opt_F',
 'opt_FAST_GEMS',
 'opt_FC',
 'opt_FC_SLICE_AX_GEMS',
 'opt_FILTERED_GEMS',
 'opt_FS',
 'opt_FT_GEMS',
 'opt_I',
 'opt_IDEAL_GEMS',
 'opt_MP_GEMS',
 'opt_MRCP_GEMS',
 'opt_NPW',
 'opt_P',
 'opt_PFF',
 'opt_PFP',
 'opt_RTR_GEMS',
 'opt_S',
 'opt_SAT2',
 'opt_SAT_GEMS',
 'opt_SEQ_GEMS',
 'opt_SFS',
 'opt_SP',
 'opt_SS_GEMS',
 'opt_T',
 'opt_TRF_GEMS',
 'opt_VASCTOF_GEMS',
 'opt_W',
 'opt_X'

In [29]:
train_label_df = train[['fname_x', 'filename', 'Parent_folder', 'patientID', 'exam', 'series', 'SeriesDescription_y', 'plane', 'seq_label', 'contrast', 'original_label_code', 'ap_label_code']]
train_label_df.rename(columns={'SeriesDescription_y':'SeriesDescription'}, inplace=True)
train_label_df

<ipython-input-29-c38f9507af25>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_label_df.rename(columns={'SeriesDescription_y':'SeriesDescription'}, inplace=True)


,fname_x,filename,Parent_folder,patientID,exam,series,SeriesDescription,plane,seq_label,contrast,original_label_code,ap_label_code
0,Abdominal MRI data/104/exam1/18 (35-68)/0038.dcm,0038.dcm,Abdominal MRI data,104,exam1,18(35-68),ax diff_TRACEW,ax,dwi,0,8,8
1,Abdominal MRI data/104/exam1/18 (1-34)/0010.dcm,0010.dcm,Abdominal MRI data,104,exam1,18(1-34),ax diff_TRACEW,ax,dwi,0,8,8
2,Abdominal MRI data/104/exam1/20/0010.dcm,0010.dcm,Abdominal MRI data,104,exam1,20,ax equilibrium,ax,equilibrium,1,16,16
3,Abdominal MRI data/104/exam1/9/0010.dcm,0010.dcm,Abdominal MRI data,104,exam1,9,ax haste bh,ax,t2,0,9,9
4,Abdominal MRI data/104/exam1/11/0010.dcm,0010.dcm,Abdominal MRI data,104,exam1,11,ax t2 triggerred,ax,t2,0,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...
1729,Abdominal MRI data/094/exam1/3/0010.dcm,0010.dcm,Abdominal MRI data,94,exam1,3,cor haste,cor,t2,0,7,7
1730,Abdominal MRI data/094/exam1/12/0010.dcm,0010.dcm,Abdominal MRI data,94,exam1,12,ax dyn,ax,early dynamic,1,2,2
1731,Abdominal MRI data/094/exam1/13/0010.dcm,0010.dcm,Abdominal MRI data,94,exam1,13,ax dyn,ax,early dynamic,1,2,2
1732,Abdominal MRI data/094/exam1/5/0010.dcm,0010.dcm,Abdominal MRI data,94,exam1,5,ax dixon_opp,ax,opposed phase,0,19,19


In [31]:
test_label_df = test[['fname_x', 'filename', 'Parent_folder', 'patientID', 'exam', 'series', 'SeriesDescription_y', 'plane', 'seq_label', 'contrast', 'original_label_code', 'ap_label_code']]
test_label_df.rename(columns={'SeriesDescription_y':'SeriesDescription'}, inplace=True)
test_label_df

<ipython-input-31-ddfd04e16e61>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_label_df.rename(columns={'SeriesDescription_y':'SeriesDescription'}, inplace=True)


,fname_x,filename,Parent_folder,patientID,exam,series,SeriesDescription,plane,seq_label,contrast,original_label_code,ap_label_code
0,Abdominal MRI data/102/exam1/16 (1-40)/0010.dcm,0010.dcm,Abdominal MRI data,102,exam1,16(1-40),ax diff_TRACEW,ax,dwi,0,8,8
1,Abdominal MRI data/102/exam1/16 (41-80)/0073.dcm,0073.dcm,Abdominal MRI data,102,exam1,16(41-80),ax diff_TRACEW,ax,dwi,0,8,8
2,Abdominal MRI data/102/exam1/18/0010.dcm,0010.dcm,Abdominal MRI data,102,exam1,18,ax equilibrium new,ax,equilibrium,1,16,16
3,Abdominal MRI data/102/exam1/11/0010.dcm,0010.dcm,Abdominal MRI data,102,exam1,11,ax dyn new,ax,early dynamic,1,3,2
4,Abdominal MRI data/102/exam1/7/0010.dcm,0010.dcm,Abdominal MRI data,102,exam1,7,ax dixon_W,ax,unknown,0,25,25
...,...,...,...,...,...,...,...,...,...,...,...,...
427,Abdominal MRI data/058/exam1/16 (1-36)/0010.dcm,0010.dcm,Abdominal MRI data,58,exam1,16(1-36),ax diff_TRACEW,ax,dwi,0,8,8
428,Abdominal MRI data/058/exam1/2/0010.dcm,0010.dcm,Abdominal MRI data,58,exam1,2,cor haste,cor,t2,0,7,7
429,Abdominal MRI data/058/exam1/13/0010.dcm,0010.dcm,Abdominal MRI data,58,exam1,13,ax dyn new,ax,early dynamic,1,2,2
430,Abdominal MRI data/058/exam1/5/0010.dcm,0010.dcm,Abdominal MRI data,58,exam1,5,ax dixon_in,ax,in phase,0,14,14


In [36]:
trainids = pd.read_csv('/content/AIPI540_individual_project/data/trainfiles.csv')
trainids

,Unnamed: 0,file_info,label,patientID,series,img_num
0,0,/volumes/cm7/Abdominal_MRI_dataset_split/train...,8,104,18(35-68),38
1,1,/volumes/cm7/Abdominal_MRI_dataset_split/train...,8,104,18(35-68),39
2,2,/volumes/cm7/Abdominal_MRI_dataset_split/train...,8,104,18(35-68),67
3,3,/volumes/cm7/Abdominal_MRI_dataset_split/train...,8,104,18(35-68),66
4,4,/volumes/cm7/Abdominal_MRI_dataset_split/train...,8,104,18(35-68),64
...,...,...,...,...,...,...
93509,93509,/volumes/cm7/Abdominal_MRI_dataset_split/train...,2,94,14,34
93510,93510,/volumes/cm7/Abdominal_MRI_dataset_split/train...,2,94,14,20
93511,93511,/volumes/cm7/Abdominal_MRI_dataset_split/train...,2,94,14,21
93512,93512,/volumes/cm7/Abdominal_MRI_dataset_split/train...,2,94,14,35


In [37]:
val_df = pd.read_pickle('/content/AIPI540_individual_project/data/val_dicom_data.pkl')

In [38]:
val_df

,label_code_x,predictions,patientID,series,img_label_code,GT label,prediction label,probability,original_label_code,SeriesDescription_y,plane,seq_label,contrast,label_code_y,final_prediction
0,8,8,41,16(37-72),8,dwi,dwi,0.976494,8,ax diff_TRACEW,ax,dwi,0,8,8
1,30,30,41,18,16,T1 post con,T1 post con fat sat,0.956994,16,ax equilibrium new,ax,equilibrium,1,16,16
2,30,30,41,11,3,T1 post con,T1 post con fat sat,0.956994,3,ax dyn new,ax,early dynamic,1,3,3
3,25,25,41,7,25,T1 fat sat,T1 fat sat,0.957891,25,ax dixon_W,ax,unknown,0,25,25
4,11,11,41,6,11,dixon fat,dixon fat,0.955352,11,ax dixon_F,ax,unknown,0,11,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,9,9,66,12,9,t2,t2,0.977097,9,ax t2 resp trig,ax,t2,0,9,9
350,7,7,66,2,17,t2,t2,0.914605,7,cor haste,cor,t2,0,7,7
351,11,11,66,5,11,dixon fat,dixon fat,0.834404,11,ax dixon top_F_FIL,ax,unknown,0,11,11
352,0,0,66,14,0,adc,adc,0.449541,0,ax diff_ADC,ax,dwi,0,0,0


In [ ]:
### does not have the filename colums.... will look for another
